[volver](main.ipynb)
# Fase 3: Preparación de los datos

## 1- Selección de Datos

Para nuestros objetivos, seleccionamos las variables que son más relevantes para el análisis de deportes de viento:

In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore') # apagamos los warnigs

# Cargar el conjunto de datos
weather_df = pd.read_csv('weatherAUS-fase-02.csv')

# Selección de las variables relevantes
variables_seleccionadas = ['Date', 'Location', 'WindGustSpeed', 'WindGustDir', 
                           'WindSpeed9am', 'WindSpeed3pm', 'WindDir9am', 'WindDir3pm', 
                           'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 
                           'Temp9am', 'Temp3pm', 'Rainfall', 'RainToday', 'RainTomorrow']

datos_seleccionados = weather_df[variables_seleccionadas]
datos_seleccionados.head()

,Date,Location,WindGustSpeed,WindGustDir,WindSpeed9am,WindSpeed3pm,WindDir9am,WindDir3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,Rainfall,RainToday,RainTomorrow
0,2008-12-01,Albury,44.0,W,20.0,24.0,W,WNW,71.0,22.0,1007.7,1007.1,16.9,21.8,0.6,No,No
1,2008-12-02,Albury,44.0,WNW,4.0,22.0,NNW,WSW,44.0,25.0,1010.6,1007.8,17.2,24.3,0.0,No,No
2,2008-12-03,Albury,46.0,WSW,19.0,26.0,W,WSW,38.0,30.0,1007.6,1008.7,21.0,23.2,0.0,No,No
3,2008-12-04,Albury,24.0,NE,11.0,9.0,SE,E,45.0,16.0,1017.6,1012.8,18.1,26.5,0.0,No,No
4,2008-12-05,Albury,41.0,W,7.0,20.0,ENE,NW,82.0,33.0,1010.8,1006.0,17.8,29.7,1.0,No,No


## 2- Limpieza de Datos

Tratamos los valores nulos y eliminamos las filas con datos faltantes en variables críticas. También imputamos valores en otras variables según sea necesario:

In [6]:
# Eliminación de filas con valores nulos en variables críticas
datos_limpios = datos_seleccionados.dropna(subset=['WindGustSpeed', 'WindGustDir'])

# Imputación de valores nulos en otras variables
datos_limpios['WindSpeed9am'].fillna(datos_limpios['WindSpeed9am'].mean(), inplace=True)
datos_limpios['WindSpeed3pm'].fillna(datos_limpios['WindSpeed3pm'].mean(), inplace=True)
datos_limpios['Humidity9am'].fillna(datos_limpios['Humidity9am'].mean(), inplace=True)
datos_limpios['Humidity3pm'].fillna(datos_limpios['Humidity3pm'].mean(), inplace=True)
datos_limpios['Pressure9am'].fillna(datos_limpios['Pressure9am'].mean(), inplace=True)
datos_limpios['Pressure3pm'].fillna(datos_limpios['Pressure3pm'].mean(), inplace=True)
datos_limpios['Temp9am'].fillna(datos_limpios['Temp9am'].mean(), inplace=True)
datos_limpios['Temp3pm'].fillna(datos_limpios['Temp3pm'].mean(), inplace=True)
datos_limpios['Rainfall'].fillna(datos_limpios['Rainfall'].mean(), inplace=True)

# Verificación después de la imputación
datos_limpios.isnull().sum()

Date                0
Location            0
WindGustSpeed       0
WindGustDir         0
WindSpeed9am        0
WindSpeed3pm        0
WindDir9am       7489
WindDir3pm        937
Humidity9am         0
Humidity3pm         0
Pressure9am         0
Pressure3pm         0
Temp9am             0
Temp3pm             0
Rainfall            0
RainToday        1195
RainTomorrow        0
dtype: int64

## 3- Construcción de Datos

Podemos crear nuevas características basadas en las variables existentes que puedan ser útiles para el análisis.

In [7]:
# Crear una nueva característica que indique si hubo viento fuerte (WindGustSpeed > 50 km/h)
datos_limpios['StrongWind'] = datos_limpios['WindGustSpeed'] > 50

# Crear una nueva característica que combine la dirección del viento de las 9 am y las 3 pm
datos_limpios['WindDirChange'] = datos_limpios['WindDir9am'] + "_" + datos_limpios['WindDir3pm']

# Creamos algunas columnas para fechas
datos_limpios['Date']  = pd.to_datetime(datos_limpios['Date'])
datos_limpios['Year'] = datos_limpios['Date'].dt.year
datos_limpios['Month'] = datos_limpios['Date'].dt.month
datos_limpios['Day'] = datos_limpios['Date'].dt.day
datos_limpios['Week'] =datos_limpios['Date'].dt.isocalendar().week
# Sacamos la columna fecha
datos_limpios = datos_limpios.drop(['Date'],axis=1)
# Mostrar las primeras filas con las nuevas características
datos_limpios.head()

,Location,WindGustSpeed,WindGustDir,WindSpeed9am,WindSpeed3pm,WindDir9am,WindDir3pm,Humidity9am,Humidity3pm,Pressure9am,...,Temp3pm,Rainfall,RainToday,RainTomorrow,StrongWind,WindDirChange,Year,Month,Day,Week
0,Albury,44.0,W,20.0,24.0,W,WNW,71.0,22.0,1007.7,...,21.8,0.6,No,No,False,W_WNW,2008,12,1,49
1,Albury,44.0,WNW,4.0,22.0,NNW,WSW,44.0,25.0,1010.6,...,24.3,0.0,No,No,False,NNW_WSW,2008,12,2,49
2,Albury,46.0,WSW,19.0,26.0,W,WSW,38.0,30.0,1007.6,...,23.2,0.0,No,No,False,W_WSW,2008,12,3,49
3,Albury,24.0,NE,11.0,9.0,SE,E,45.0,16.0,1017.6,...,26.5,0.0,No,No,False,SE_E,2008,12,4,49
4,Albury,41.0,W,7.0,20.0,ENE,NW,82.0,33.0,1010.8,...,29.7,1.0,No,No,False,ENE_NW,2008,12,5,49


## 4- Integración de Datos

Usamos el conjunto de dato creado en la fase 2 con las georeferencias relevantes, las integraremos con el dataset weatherAUS-fase-02.csv en este paso. 


In [13]:
# Cargar el conjunto de datos
weather_df_geo = pd.read_csv('weatherAUS-fase-02-with-georef.csv')

# Fusionar por la característica 'Location'
df_weatherAUS_geo = pd.merge(datos_limpios, weather_df_geo[['Location','Latitude','Longitude','Is_Coastal']], on='Location')
df_weatherAUS_geo.head()

,Location,WindGustSpeed,WindGustDir,WindSpeed9am,WindSpeed3pm,WindDir9am,WindDir3pm,Humidity9am,Humidity3pm,Pressure9am,...,RainTomorrow,StrongWind,WindDirChange,Year,Month,Day,Week,Latitude,Longitude,Is_Coastal
0,Albury,44.0,W,20.0,24.0,W,WNW,71.0,22.0,1007.7,...,No,False,W_WNW,2008,12,1,49,-36.080477,146.91628,False
1,Albury,44.0,WNW,4.0,22.0,NNW,WSW,44.0,25.0,1010.6,...,No,False,NNW_WSW,2008,12,2,49,-36.080477,146.91628,False
2,Albury,46.0,WSW,19.0,26.0,W,WSW,38.0,30.0,1007.6,...,No,False,W_WSW,2008,12,3,49,-36.080477,146.91628,False
3,Albury,24.0,NE,11.0,9.0,SE,E,45.0,16.0,1017.6,...,No,False,SE_E,2008,12,4,49,-36.080477,146.91628,False
4,Albury,41.0,W,7.0,20.0,ENE,NW,82.0,33.0,1010.8,...,No,False,ENE_NW,2008,12,5,49,-36.080477,146.91628,False




## 5- Formateo de Datos

Finalmente, aseguramos que los datos estén en el formato adecuado para el análisis y el modelado. Por ejemplo, convertir variables categóricas a numéricas, normalizar valores, etc.

In [14]:
# Convertir variables categóricas a variables dummy
datos_finales = df_weatherAUS_geo.copy() #pd.get_dummies(datos_limpios, columns=['WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow'])

# Normalizar las variables numéricas
from sklearn.preprocessing import StandardScaler

escalador = StandardScaler()
variables_numericas = ['WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm', 'Rainfall']
datos_finales[variables_numericas] = escalador.fit_transform(datos_finales[variables_numericas])


# label encoding
le = LabelEncoder()
datos_finales['LocationEncoder'] = le.fit_transform(datos_finales['Location'])
datos_finales['WindDir9amEncoder'] = le.fit_transform(datos_finales['WindDir9am'])
datos_finales['WindDir3pmEncoder'] = le.fit_transform(datos_finales['WindDir3pm'])
datos_finales['WindGustDirEncoder'] = le.fit_transform(datos_finales['WindGustDir'])
datos_finales['Is_CoastalEncoder'] = le.fit_transform(datos_finales['Is_Coastal'])
datos_finales['RainTomorrowEncoder'] = le.fit_transform(datos_finales['RainTomorrow'])
datos_finales['RainTodayEncoder'] = le.fit_transform(datos_finales['RainToday'])
datos_finales['StrongWindEncoder'] = le.fit_transform(datos_finales['StrongWind'])
datos_finales['WindDirChangeEncoder'] = le.fit_transform(datos_finales['WindDirChange'])

# Mostrar las primeras filas del conjunto de datos final
datos_finales.head()

,Location,WindGustSpeed,WindGustDir,WindSpeed9am,WindSpeed3pm,WindDir9am,WindDir3pm,Humidity9am,Humidity3pm,Pressure9am,...,Is_Coastal,LocationEncoder,WindDir9amEncoder,WindDir3pmEncoder,WindGustDirEncoder,Is_CoastalEncoder,RainTomorrowEncoder,RainTodayEncoder,StrongWindEncoder,WindDirChangeEncoder
0,Albury,0.477226,W,0.824723,0.737977,W,WNW,0.078742,-1.438779,-1.462315,...,False,1,13,14,13,0,0,0,0,254
1,Albury,0.477226,WNW,-1.119200,0.491939,NNW,WSW,-1.375117,-1.289513,-1.039680,...,False,1,6,15,14,0,0,0,0,95
2,Albury,0.645102,WSW,0.703228,0.984015,W,WSW,-1.698197,-1.040737,-1.476888,...,False,1,13,15,15,0,0,0,0,255
3,Albury,-1.201533,NE,-0.268734,-1.107309,SE,E,-1.321271,-1.737309,-0.019527,...,False,1,9,0,4,0,0,0,0,128
4,Albury,0.225413,W,-0.754714,0.245901,ENE,NW,0.671055,-0.891472,-1.010533,...,False,1,1,7,13,0,0,0,0,7


In [15]:
# Guardar el DataFrame en un archivo CSV para usar en la fase 4
datos_finales.to_csv('weatherAUS-fase-03.csv', index=False)

## Resumen de la Preparación de los Datos

### Selección de Datos:

* Seleccionamos las variables más relevantes para el análisis de deportes de viento.

### Limpieza de Datos:

* Tratamos los valores nulos eliminando filas y realizando imputaciones.

### Construcción de Datos:

* Creamos nuevas características derivadas de las variables existentes.

### Integración de Datos:

* (No se realizó integración adicional en este caso).

### Formateo de Datos:

* Convertimos variables categóricas a variables dummy y normalizamos las variables numéricas.

## Informe de Preparación de los Datos

**Objetivo:** Preparar los datos meteorológicos para el análisis y modelado de deportes de viento.

### Selección de Datos:

* Variables seleccionadas relacionadas con el viento y otras condiciones climáticas.

### Limpieza de Datos:

* Eliminación de filas con valores nulos en variables críticas.
* Imputación de valores nulos en otras variables.

### Construcción de Datos:

* Creación de nuevas características como StrongWind y WindDirChange.

### Integración de Datos:

* No se realizó integración adicional.

### Formateo de Datos:

* Conversión de variables categóricas a variables dummy.
* Normalización de variables numéricas.

### Conclusión:

* Los datos están listos para la fase de modelado y análisis avanzado.